#Import the required libraries




In [1]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

# Loading Documents


In [2]:
loader = PyPDFLoader('https://www.nestle.com/sites/default/files/asset-library/documents/jobs/the_nestle_hr_policy_pdf_2012.pdf')
documents = loader.load()

# initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Maximum size of each chunk
    chunk_overlap=200 # Overlap between chunks to preserve context
)

# Split the loaded documents
split_documents = text_splitter.split_documents(documents)

# verify the split documents
print(f"Number of split documents: {len(split_documents)}")
# preview the last 3 chunks
for i, doc in enumerate(split_documents[-3:]):  # Preview last 3 chunks
    print(f"Chunk {i+1}: {doc.page_content[:200]}...")  # Print first 200 characters of each chunk


Number of split documents: 20
Chunk 1: that goes beyond the traditional aspects of 
collective bargaining in order to share knowledge 
and to jointly find opportunities related to 
important matters such as Creating Shared Value, 
the heal...
Chunk 2: minimal levels of management and broad spans 
of control, which enable people development, 
increase efficiency, and ease implementation 
of our “Nestlé Management and Leadership 
Principles”.
Less hi...
Chunk 3: A dynamic organisation creates a climate 
of innovation and allows people to think from 
different perspectives. At Nestlé we encourage 
our people to take risks. Mistakes may be made 
but there is al...


 # Creating Vector Representation of Texts

In [3]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# number of chunks for cheaper embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)




C:\Users\Abibou\AppData\Local\Temp\ipykernel_9572\2926562203.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


# Setting Up Question-Answering System

In [4]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())
query = "When was the Nestlé HR Policy last updated?"

result = qa.invoke(query)
print("Query:", query)
print("Answer:", result['result'])



Query: When was the Nestlé HR Policy last updated?
Answer: The Nestlé HR Policy was last updated in September 2012.


# Defining Prompt Template

In [5]:
from langchain import PromptTemplate

# Define the prompt template in English
template = """
I am a HR helpful assistant. Please answer the following question in English.
Question: {question}
Answer:
"""

# Create the PromptTemplate instance with the modified English template
prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)




#  Building Chat Interface with Gradio and Launching the Chat Interface

In [6]:
import gradio as gr



def add_text(history, text):
    history = history + [{"role": "user", "content": text}]
    return history, ""

def bot(history):
    query = history[-1]["content"]
    query = prompt.format(question=query)
    answer = qa.run(query)
    # You can add source info if needed
    history[-1]["content"] = answer
    history[-1]["role"] = "assistant"
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        value=[],
        elem_id="chatbot",
        type="messages",
        height=400
    )

    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
            show_label=False,
            placeholder="Enter text and press enter",
            container=False
        )

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch(share=True)

c:\Python311\Lib\site-packages\gradio\layouts\column.py:59: UserWarning: 'scale' value should be an integer. Using 0.6 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ead086e1db263f7fcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\Abibou\AppData\Local\Temp\ipykernel_9572\1580668044.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)
